# Подготовка среды

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [4]:
folder = "bert-base-multilingual-cased-liter"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

In [5]:
!pip install transformers
!pip install razdel

     |████████████████████████████████| 2.1MB 15.8MB/s 
     |████████████████████████████████| 901kB 42.7MB/s 
     |████████████████████████████████| 3.3MB 42.2MB/s 


In [6]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [7]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [8]:
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [9]:
def my_dump(file, name):
    with open("./bin/" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./bin/" + name + ".pic", 'rb') as f:
        return pc.load(f)

#text processing

## generate (split) sentences

In [8]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [10]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 20.4 s, sys: 5.36 s, total: 25.8 s
Wall time: 31.5 s


In [11]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

6287032

##process

In [12]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [13]:
%%time
batch = 1000
start_time = time.time()

ftom_b = 983
to_b = 1000

print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

for j in range(ftom_b, to_b):
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha() and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

senteces from 983000 to 1000000
done 00983 --- time: 136.46670031547546	--- total: 136.46736478805542
done 00984 --- time: 121.48819851875305	--- total: 257.96250915527344
done 00985 --- time: 127.26888060569763	--- total: 385.2370924949646
done 00986 --- time: 122.53102421760559	--- total: 507.7730977535248
done 00987 --- time: 133.3560791015625	--- total: 641.1343500614166
done 00988 --- time: 134.8227150440216	--- total: 775.9620938301086
done 00989 --- time: 143.0957691669464	--- total: 919.0638751983643
done 00990 --- time: 139.16708540916443	--- total: 1058.2375173568726
done 00991 --- time: 124.49693298339844	--- total: 1182.7409167289734
done 00992 --- time: 125.44592928886414	--- total: 1308.1939063072205
done 00993 --- time: 129.44963574409485	--- total: 1437.6481113433838
done 00994 --- time: 133.41076064109802	--- total: 1571.065125465393
done 00995 --- time: 116.42566847801208	--- total: 1687.4996943473816
done 00996 --- time: 118.51502847671509	--- total: 1806.01952004432

#aggregation of embeddings



In [14]:
folder = "bert-base-multilingual-cased-liter"

##list of files

In [15]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [16]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 5min 24s, sys: 15.3 s, total: 5min 39s
Wall time: 18min 33s


In [17]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

[('как', 64780),
 ('про', 64509),
 ('что', 63156),
 ('the', 62882),
 ('при', 60436),
 ('его', 45504),
 ('под', 43608),
 ('это', 34364),
 ('раз', 34024),
 ('пер', 28503),
 ('для', 28195),
 ('and', 24534),
 ('стр', 23110),
 ('все', 22707),
 ('пред', 21919),
 ('так', 21326),
 ('образ', 20005),
 ('пре', 19668),
 ('лит', 18327),
 ('поэт', 17147)]

In [18]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

NameError: ignored

In [ ]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [ ]:
batch = 5000
start_time = time.time()

ftom_b = 0
to_b = 10

print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

all_emb = {}
for b in range(ftom_b, to_b):
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

#building dictionary


In [ ]:
folder = "bert-base-multilingual-cased-liter"

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [ ]:
with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
    json.dump(lista_items, f, indent=4)

#Tests

##preparation

In [ ]:
folder = "bert-base-multilingual-cased-liter"

In [30]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [36]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])
def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [ ]:
# get_word_m(dct1["машина"], dct1, 10)

##GATS

###generate test

In [47]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [ ]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [ ]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [ ]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [ ]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [ ]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [14]:
with open("tests/gats/ready.pic", "br") as f:
    words_rus = pc.load( f)

In [24]:
def cut_b(text):
    return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]

###3cosadd

In [34]:
def cos_add(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                m.append(cut_b(i))
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [37]:
res_df1 = cos_add(dct1)

In [42]:
# res_df1[res_df1["group"] == ": gram2-opposite"]

In [40]:
res_df1["group"].unique()

array([': capital-world', ': family', ': gram1-adjective-to-adverb',
       ': gram2-opposite', ': gram3-comparative',
       ': gram6-nationality-adjective'], dtype=object)

In [39]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.793130,0.000000,20
Comparative,0.845331,0.206522,552
Adjective adverb,0.857969,0.222222,306
Nationality adjective,0.899240,0.333333,6
Family,0.917592,0.659091,132
Capital-countries,0.832437,0.666667,3


In [55]:
words_rus

{': capital-world': [['абуджа', 'нигерия', 'аккра', 'гана'],
  ['абуджа', 'нигерия', 'алжир', 'алжир'],
  ['абуджа', 'нигерия', 'амман', 'иордания'],
  ['абуджа', 'нигерия', 'анкара', 'турция'],
  ['абуджа', 'нигерия', 'антананариву', 'мадагаскар'],
  ['абуджа', 'нигерия', 'апиа', 'самоа'],
  ['абуджа', 'нигерия', 'ашхабад', 'туркменистан'],
  ['абуджа', 'нигерия', 'асмара', 'эритрея'],
  ['абуджа', 'нигерия', 'астана', 'казахстан'],
  ['абуджа', 'нигерия', 'афины', 'греция'],
  ['абуджа', 'нигерия', 'багдад', 'ирак'],
  ['абуджа', 'нигерия', 'баку', 'азербайджан'],
  ['абуджа', 'нигерия', 'бамако', 'мали'],
  ['абуджа', 'нигерия', 'бангкок', 'таиланд'],
  ['абуджа', 'нигерия', 'банджул', 'гамбия'],
  ['абуджа', 'нигерия', 'пекин', 'китай'],
  ['абуджа', 'нигерия', 'бейрут', 'ливан'],
  ['абуджа', 'нигерия', 'белград', 'сербия'],
  ['абуджа', 'нигерия', 'бельмопан', 'белиз'],
  ['абуджа', 'нигерия', 'берлин', 'германия'],
  ['абуджа', 'нигерия', 'берн', 'швейцария'],
  ['абуджа', 'ниге

In [58]:
cut_b("камбоджа")

'кам'